# Edgar Q&A Demo

In [1]:
import os
import sys
from pprint import pprint

current_dir = os.getcwd()
kit_dir = os.path.abspath(os.path.join(current_dir, ".."))
repo_dir = os.path.abspath(os.path.join(kit_dir, ".."))

sys.path.append(kit_dir)
sys.path.append(repo_dir)

from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA, LLMChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryBufferMemory, ConversationSummaryMemory, ChatMessageHistory
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)

from langchain_community.llms.sambanova import SambaStudio, Sambaverse
from utils.model_wrappers.langchain_llms import SambaNovaFastAPI

from dotenv import load_dotenv
load_dotenv(os.path.join(repo_dir,'.env'))

True

In [2]:
PERSIST_DIRECTORY = os.path.join(kit_dir,'data/vectordbs/tsla')

### Load embedding model and SEC vector db

In [3]:
encode_kwargs = {"normalize_embeddings": True}
embedding_model = HuggingFaceInstructEmbeddings(
    model_name='intfloat/e5-large-v2',
    embed_instruction="",  # no instruction is needed for candidate passages
    query_instruction="Represent the query for retrieval: ",
    encode_kwargs=encode_kwargs,

)
vectordb = Chroma(persist_directory=PERSIST_DIRECTORY, embedding_function=embedding_model)

/Users/petrojm/Documents/projects/ASK/fastAPI/ai-starter-kit/edgar_qna/edgar_fastapi/lib/python3.10/site-packages/InstructorEmbedding/instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


load INSTRUCTOR_Transformer
max_seq_length  512


/Users/petrojm/Documents/projects/ASK/fastAPI/ai-starter-kit/edgar_qna/edgar_fastapi/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


### Choose LLM

In [15]:
# Using SambaStudio
#llm = SambaStudio(
#        streaming=True,
#        model_kwargs={
#            'max_tokens_to_generate': 512,
#            'select_expert': 'Meta-Llama-3-8B-Instruct',
#            'temperature': 0.0,
#            'repetition_penalty': 1.0,
#            'top_k': 1,
#            'top_p': 1.0,
#            'do_sample': False,
#        },
#    )

# Using Sambaverse
llm = Sambaverse(
     sambaverse_model_name='Meta/llama-2-70b-chat-hf',
     model_kwargs={
         'do_sample': False,
         'max_tokens_to_generate': 512,
         'select_expert': 'llama-2-70b-chat-hf',
         'process_prompt': False,
         'temperature': 0.01,
     },
 )

# Using SambaNovaFastAPI 
#llm = SambaNovaFastAPI(
#        max_tokens = 512,
#        model= 'llama3-8b',
#)


### Retrieve information from vector db

In [16]:
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

qa = RetrievalQA.from_chain_type(
    llm=llm,
    #chain_type="stuff",
    retriever=retriever,
    input_key="question",
    output_key="response",
    return_source_documents=True,
)

custom_prompt_template = """
<s>[INST] <<SYS>>\nYou're an expert in filing reports\n<</SYS>>\n 
Given the following context enclosed in backticks regarding a company annual/quarterly report filing:
```
{context}
```
Consider the question:  
{question}
Answer the question using only the information from the context. If the answer to the question can't be extracted from the preovious context, then say "I do not have information regarding this".
Helpful Answer: [/INST]"""
CUSTOMPROMPT = PromptTemplate(
    template=custom_prompt_template, input_variables=["context", "question"]
)
## Inject custom prompt
qa.combine_documents_chain.llm_chain.prompt = CUSTOMPROMPT
query = "What's the name of the company of this filing report?"
response = qa({"question": query})
print(response['response'])

 The name of the company in the filing report is Tesla, Inc. This can be found in the last line of the context provided: "Commission File Number: 001-34756>> Tesla, Inc."


### Q&A chatbot

The following image shows the general idea of the architecture for this retriever chatbot. </br>
More resources about how chatbots are implemented with langchain [here](https://python.langchain.com/docs/use_cases/chatbots)

![retriever_chatbot_architecture](https://python.langchain.com/assets/images/chat_use_case-eb8a4883931d726e9f23628a0d22e315.png )

### Chatbot example

In [7]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history."
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("<s>[INST] {question} [/INST]"),
    ]
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
conversation = LLMChain(llm=llm, prompt=prompt, verbose=True, memory=memory)

conversation({"question": "hi, my name is Rodrigo"})

/Users/petrojm/Documents/projects/ASK/fastAPI/ai-starter-kit/edgar_qna/edgar_fastapi/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(




> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history.
Human: <s>[INST] hi, my name is Rodrigo [/INST]

> Finished chain.


{'question': 'hi, my name is Rodrigo',
 'chat_history': [HumanMessage(content='hi, my name is Rodrigo'),
  AIMessage(content=' <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[

In [8]:
conversation({"question": "can you tell me about the benefits of using AI in society?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history.
Human: hi, my name is Rodrigo
AI:  <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[

{'question': 'can you tell me about the benefits of using AI in society?',
 'chat_history': [HumanMessage(content='hi, my name is Rodrigo'),
  AIMessage(content=' <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <

In [9]:
conversation({"question": "could you tell me more details about point 1?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history.
Human: hi, my name is Rodrigo
AI:  <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[

{'question': 'could you tell me more details about point 1?',
 'chat_history': [HumanMessage(content='hi, my name is Rodrigo'),
  AIMessage(content=' <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>

In [10]:
conversation({"question": "Great, what was my name?"})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful assistant. Answer questions with the best of your capabilities and based on the chat history.
Human: hi, my name is Rodrigo
AI:  <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[

{'question': 'Great, what was my name?',
 'chat_history': [HumanMessage(content='hi, my name is Rodrigo'),
  AIMessage(content=' <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s>[/INST] <s

### Chatbot with Summary and SEC retriever

In [11]:
def print_response(response: dict) -> None:
    
    print('\nQUESTION:')
    print(response['question'])
    
    print('\nCONTEXT:')
    for document in response['source_documents']:
        pprint(document.page_content, width=180)
        print()
    
    print('\nCHAT HISTORY:')
    for message in response['chat_history']:
        message.pretty_print()
        
    print('\nANSWER:')
    print(response['answer'])

In [12]:
custom_condensed_question_template = """<s>[INST] <<SYS>>\nYou're an assitant\n<</SYS>>
Given the following chat history and follow up question, rephrase the follow up question using the chat history. Only output the rephrased question.

Chat history:
{chat_history}

Follow up question: {question}

Rephrased question: [/INST]"""

custom_condensed_question_prompt = PromptTemplate.from_template(custom_condensed_question_template)

custom_qa_template = """
<s>[INST] <<SYS>>\nYou're a helpful assistant in filing reports. Follow the following rules:
1. If you don't know the answer, respond kindly that you don't have information about it. 
2. Do not try to make up an answer.
3. Start the conversation expressing general assitance in anything the user would like to ask.\n<</SYS>>
Given the following context and follow up question, provide a helpful answer.

Context:
{context}

Follow up question: {question}

Helpful answer: [/INST]"""

custom_qa_prompt = PromptTemplate.from_template(custom_qa_template)

memory = ConversationSummaryMemory(
    llm=llm, 
    max_token_limit=50,
    buffer="The human and AI greet each other.",
    output_key='answer',
    memory_key='chat_history',
    return_messages=True,
)

retriever = vectordb.as_retriever(search_kwargs={"k": 3})

qa = ConversationalRetrievalChain.from_llm(
    llm, 
    retriever=retriever, 
    memory=memory, 
    chain_type="stuff",
    return_source_documents=True, 
    verbose=True,
    condense_question_prompt = custom_condensed_question_prompt,
    combine_docs_chain_kwargs={'prompt': custom_qa_prompt}
)

response = qa({"question": "what are the biggest risk that the company is facing?"})
print_response(response)



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You're an assitant
<</SYS>>
Given the following chat history and follow up question, rephrase the follow up question using the chat history. Only output the rephrased question.

Chat history:

system: The human and AI greet each other.

Follow up question: what are the biggest risk that the company is facing?

Rephrased question: [/INST]

> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

<s>[INST] <<SYS>>
You're a helpful assistant in filing reports. Follow the following rules:
1. If you don't know the answer, respond kindly that you don't have information about it. 
2. Do not try to make up an answer.
3. Start the conversation expressing general assitance in anything the user would like to ask.
<</SYS>>
Given the following context and follow up question, provide a helpful answer.

Context:
action, require us to verify the correctness 

In [13]:
response = qa({"question": "Tell me about the business of the company, what products and services does it offer?"})
print_response(response)



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You're an assitant
<</SYS>>
Given the following chat history and follow up question, rephrase the follow up question using the chat history. Only output the rephrased question.

Chat history:

system:  The human and AI greet each other. The human asks what the biggest risk that the company is facing. The AI identifies several risks, including liability under laws and regulations, reliance on service providers, financial performance risks, and potential impairment of assets. The AI provides a detailed analysis of the risks, highlighting the potential impact on the company's financial performance and reputation. The AI's response is a lengthy and detailed analysis of the company's risks, providing a comprehensive overview of the potential challenges the company may face. The AI's response is a lengthy and detailed analysis of the company's risks, providing a comprehensive overview of the potential challenges th

In [14]:
response = qa({"question": "tell me more about the products that Tesla offers based on the report"})
print_response(response)



> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
You're an assitant
<</SYS>>
Given the following chat history and follow up question, rephrase the follow up question using the chat history. Only output the rephrased question.

Chat history:

system:  The human and AI greet each other. The human asks what the biggest risk that the company is facing. The AI identifies several risks, including liability under laws and regulations, reliance on service providers, financial performance risks, and potential impairment of assets. The AI provides a detailed analysis of the risks, highlighting the potential impact on the company's financial performance and reputation. The human asks about the company's business operations, and the AI clarifies that the financial statements do not explicitly mention the company's products and services. The AI offers to help analyze the financial statements to identify potential areas of risk and provide insights into the company's ope